In [1]:
# אז מה צריך לעשות ?
# כל תמונה דבר ראשון רצים עליה ומסווגים לאיזה סוג היא:
# תמונה- אז רק מכניסים את הנתיב שלה  לDB לטבלת תמונות
# אם זה כירכת יד ושם- אז שולחים לפונקציה ש 

In [2]:
# pip install matplotlib

In [3]:
# pip install opencv-python

In [4]:
'C:\Python310\python.exe -m pip install --upgrade pip'

'C:\\Python310\\python.exe -m pip install --upgrade pip'

In [5]:
# pip install joblib

In [6]:
# pip install scikit-image
# # #

In [7]:
# pip install scikit-learn

In [8]:
# pip install --upgrade --force-reinstall tensorflow


In [9]:
# !pip show tensorflow

In [10]:
# pip install pyodbc

In [11]:
import cv2
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.patches as mpatches
import os, os.path
from matplotlib.patches import Rectangle
from PIL import Image
import glob
import base64
import requests
import json
import io
import ast
import numpy as np
import joblib
import ast
from skimage.transform import resize
import PIL
import tensorflow as tf
import pyodbc
import json

In [12]:
import spacy

In [13]:
# !pip install spacy
# !python -m spacy download en_core_web_sm
# !python -m spacy download en_core_web_trf
# !python -m spacy download en_core_web_trf

In [14]:
# !python -m spacy download en_core_web_sm

In [15]:
# !python -m spacy download en_core_web_trf

In [16]:
def get_normal_text(i):
    if i:
    # Check and replace '.' if it exists in the string
        if '.' in i:
            i = i.replace('.', '')
    
        # Check and replace '/' if it exists in the string
        if '/' in i:
            i = i.replace('/', '')
    
        # Check and replace ',' if it exists in the string
        if ',' in i:
            i = i.replace(',', '')
    
        # Check and replace ':' if it exists in the string
        if ':' in i:
            i = i.replace(':', '')
    
        # Check and replace ')' if it exists in the string
        if ')' in i:
            i = i.replace(')', '')

    # Check and replace '(' if it exists in the string
        if '(' in i:
            i = i.replace('(', '')

        return i

In [51]:
def get_finds_dates_places_persons(desc):
    nlp = spacy.load("en_core_web_trf")
    doc = nlp(desc)
    entity_types = {}
    
    for ent in doc.ents:
        type_name = ent.label_
        entity = ent.text

        if type_name not in entity_types:
            entity_types[type_name] = []

        entity_types[type_name].append(entity)
    target_word=nlp("study")
    return entity_types

In [18]:
def get_num_words_in_json(json):
    textAnnotations = json['responses'][0]['textAnnotations']
    word_count=0
    for word in textAnnotations:
        desc = word['description']
        word_count+=1
    return word_count

In [19]:
def get_all_words_with_vertices(data_response):
    textAnnotations=data_response['responses'][0]["textAnnotations"]
    text=data_response['responses'][0]['fullTextAnnotation']['text']
    arr_res=[]
    arr_res.append(({'text':text},textAnnotations[0]['boundingPoly'] ))
    for idx, i in enumerate(textAnnotations, start=1):
        ver=i["boundingPoly"]
        desc=i['description']
        desc=desc.split("\n")
        list1=(ver,desc)
        arr_res.append(list1)
    xx=arr_res[0]
    arr_res= arr_res[1:]
    return [arr_res, xx]

In [20]:
def create_json_get_by_y(json):
    [words,full_desc_with_verices] =get_all_words_with_vertices(json)
    json_res={}
    json_res["full_desc_with_verices"]=full_desc_with_verices
    json_res['words']=words
    return json_res

In [21]:
def get_size_y(json):
    vertices = json["full_desc_with_verices"][1]["vertices"]
    y_start = vertices[0].get("y", 0)  # default value 0 if 'y' key is not found in the dictionary
    y_end = vertices[3].get("y", 0)
    
    x_start = vertices[0].get("x", 0)  # default value 0 if 'y' key is not found in the dictionary
    x_end = vertices[1].get("x", 0)
    if "y" in vertices[0] and "y" in vertices[3]:
        range_y = vertices[3]["y"] - vertices[0]["y"]
    else:
        range_y = 0  # or set a default value if 'y' key is missing in some vertices
    if "x" in vertices[1] and "x" in vertices[0]:
        range_x= vertices[1]["x"] - vertices[0]["x"]
    else:
        range_x = 0  # or set a default value if 'y' key is missing in some vertices
    
    
    if range_y != 0 and range_x!=0:
        loc_title_up= (y_start,y_start+range_y/4)
        loc_title_middle=(y_start+range_y/4,y_start+(range_y/3)*2)
        loc_title_down=(y_start+(range_y/3)*2,y_start+range_y)
        x_middle=(x_start+(range_x/2))
        return [loc_title_up,loc_title_middle,loc_title_down, x_middle]
    else:
        return [(0, 0), (0, 0), (0,0), (0, 0)]

In [22]:
def get_text_from_block(block):
    text=''
    words=block['paragraphs'][0]['words']
    for w in words:
        word=w['symbols']
        for l in word:
            text+=l['text']
        text+=' '
    return text

In [23]:
def get_big_letter_from_desc(json, words_count):
    text_annotations = json.get('responses', [{}])[0].get('textAnnotations', [])
    
    block_areas_num = []
    
    for word in text_annotations[1:]:
        desc = word.get('description', '')
        block_vertices = word.get('boundingPoly', {}).get('vertices', [])
        x_values = [vertex.get('x', 0) for vertex in block_vertices if 'x' in vertex]
        y_values = [vertex.get('y', 0) for vertex in block_vertices if 'y' in vertex]
        
        if len(x_values) > 0 and len(y_values) > 0:
            max_x = max(x_values)
            min_x = min(x_values)

            max_y = max(y_values)
            min_y = min(y_values)

            block_x = max_x - min_x
            block_y = max_y - min_y

            num_letter = len(desc)
            area = block_x * block_y
            letter = area / num_letter
            
            if desc not in ['(', ')', '-', '[', ']', ',', '.']:
                block_areas_num.append((desc, letter))
    
    sorted_block_areas_num = sorted(block_areas_num, key=lambda x: x[1], reverse=True)
    
    return sorted_block_areas_num

In [24]:
def get_date_Testimony(json_res, json, date_arr, format_json):
    res_date=''
    side_date=1
    data_Language=format_json['data_Languages'][0][0]
    if data_Language=='Hebrew' or data_Language=='Arabic':
        side_date=0
    [loc_title_up,loc_title_middle,loc_title_down, x_middle ]=get_size_y(json_res)
    print(loc_title_up)
    if [loc_title_up,loc_title_middle,loc_title_down, x_middle ]!=[(0, 0), (0, 0), (0,0), (0, 0)]:
        fullTextAnnotation_pages = json['responses'][0]['fullTextAnnotation']['pages']
        dict_block_by_loc_date={0:[[], []], 1:[[], []], 2:[[], []]}
        for page in fullTextAnnotation_pages:
            blocks = page['blocks']
            blocks=sorted( blocks, key=lambda block:block['boundingBox']['vertices'][0]['y'] )
            for block in blocks:
                # 0 left
                # 1 right
                side=0
                text_from_block=get_text_from_block(block)
                words=block['paragraphs'][0]['words']
                block_vertices = block['boundingBox']['vertices']
                y=block_vertices[3]['y']-block_vertices[0]['y']
                x=block_vertices[0]['x']
                if x>=x_middle:
                    side=1
                if block_vertices[0]['y']<=loc_title_up[1]:
                    ans=0
                elif block_vertices[0]['y']<=loc_title_middle[1]:
                    ans=1
                else:
                    ans=2
                desc_words=''
                for word in words:
                    word_vertices=word['boundingBox']['vertices'] 
                    desc=''
                    word1=word['symbols']
                    for l in word1:
                        desc+=l['text']
                    desc_words+=desc
                    desc_words+=' '
                    for date in date_arr:
                        date_split=date.split(' ')
                        for ds in date_split:
                            if desc==ds:
                                dict_block_by_loc_date[ans][side].append(date)
    if dict_block_by_loc_date[2]:
        arr=dict_block_by_loc_date[2]
        res_arr=arr[side_date]
        if len(res_arr)>0:
            return res_arr[0]
        else:
            res_arr=arr[abs(side_date-1)]
            if len(res_arr)>0:
                return res_arr[0]
    # elif dict_block_by_loc_date[2]:
    #     arr=dict_block_by_loc_date[2]
    #     res_arr=arr[side_date]
    #     if len(res_arr)>0:
    #         return res_arr[0]
    #     else:
    #         res_arr=arr[abs(side_date-1)]
    #         if len(res_arr)>0:
    #             return res_arr[0]
    # elif dict_block_by_loc_date[1]:
    #     arr=dict_block_by_loc_date[1]
    #     res_arr=arr[side_date]
    #     if len(res_arr)>0:
    #         return res_arr[0]
    #     else:
    #         res_arr=arr[abs(side_date-1)]
    #         if len(res_arr)>0:
    #             return res_arr[0]
    return '' 

In [25]:
def get_date(json_res, json, date_arr, format_json):
    res_date=''
    side_date=1
    data_Language=format_json['data_Languages'][0][0]
    if data_Language=='Hebrew' or data_Language=='Arabic':
        side_date=0
    [loc_title_up,loc_title_middle,loc_title_down, x_middle ]=get_size_y(json_res)
    print(loc_title_up)
    if [loc_title_up,loc_title_middle,loc_title_down, x_middle ]!=[(0, 0), (0, 0), (0,0), (0, 0)]:
        fullTextAnnotation_pages = json['responses'][0]['fullTextAnnotation']['pages']
        dict_block_by_loc_date={0:[[], []], 1:[[], []], 2:[[], []]}
        for page in fullTextAnnotation_pages:
            blocks = page['blocks']
            blocks=sorted( blocks, key=lambda block:block['boundingBox']['vertices'][0]['y'] )
            for block in blocks:
                # 0 left
                # 1 right
                side=0
                text_from_block=get_text_from_block(block)
                words=block['paragraphs'][0]['words']
                block_vertices = block['boundingBox']['vertices']
                y=block_vertices[3]['y']-block_vertices[0]['y']
                x=block_vertices[0]['x']
                if x>=x_middle:
                    side=1
                if block_vertices[0]['y']<=loc_title_up[1]:
                    ans=0
                elif block_vertices[0]['y']<=loc_title_middle[1]:
                    ans=1
                else:
                    ans=2
                desc_words=''
                for word in words:
                    word_vertices=word['boundingBox']['vertices'] 
                    desc=''
                    word1=word['symbols']
                    for l in word1:
                        desc+=l['text']
                    desc_words+=desc
                    desc_words+=' '
                    for date in date_arr:
                        date_split=date.split(' ')
                        for ds in date_split:
                            if desc==ds:
                                dict_block_by_loc_date[ans][side].append(date)
    if dict_block_by_loc_date[0]:
        arr=dict_block_by_loc_date[0]
        res_arr=arr[side_date]
        if len(res_arr)>0:
            return res_arr[0]
        else:
            res_arr=arr[abs(side_date-1)]
            if len(res_arr)>0:
                return res_arr[0]
    elif dict_block_by_loc_date[2]:
        arr=dict_block_by_loc_date[2]
        res_arr=arr[side_date]
        if len(res_arr)>0:
            return res_arr[0]
        else:
            res_arr=arr[abs(side_date-1)]
            if len(res_arr)>0:
                return res_arr[0]
    elif dict_block_by_loc_date[1]:
        arr=dict_block_by_loc_date[1]
        res_arr=arr[side_date]
        if len(res_arr)>0:
            return res_arr[0]
        else:
            res_arr=arr[abs(side_date-1)]
            if len(res_arr)>0:
                return res_arr[0]
    return '' 

In [26]:
def get_title(json,json_res, arr_date):
    [loc_title_up,loc_title_middle,loc_title_down ]=get_size_y(json_res)
    if [loc_title_up,loc_title_middle,loc_title_down ]!=[(0, 0), (0, 0), (0,0,)]:
        fullTextAnnotation_pages = json['responses'][0]['fullTextAnnotation']['pages']
        word_count=get_num_words_in_json(json) 
        sorted_block_areas_num=get_big_letter_from_desc(json,word_count )
        dict_block_by_loc_title={0:[], 1:[], 2:[]}
        dict_mean_letter_in_block=[]
        private_ans=0
        for page in fullTextAnnotation_pages:
            blocks = page['blocks']
            if (len(blocks)<=3):
                private_ans=2
            for block in blocks:
                words=block['paragraphs'][0]['words']
                len_words=len(words)
                if len_words<=15:
                    mean_letter_in_block=0
                    text=get_text_from_block(block)
                    block_vertices = block['boundingBox']['vertices']
                    y=block_vertices[3]['y']-block_vertices[0]['y']
                    if block_vertices[0]['y']<=loc_title_up[1]:
                        ans=1
                    elif block_vertices[0]['y']<=loc_title_middle[1]:
                        ans=2
                    else:
                        ans=3
                    for word in words:
                        desc=''
                        word1=word['symbols']
                        for l in word1:
                            desc+=l['text']
                        word_vertices=word['boundingBox']['vertices']
                        x_values = []
                        for vertex in block_vertices:
                            if 'x' in vertex:
                                x_values.append(vertex['x'])
                        max_x = max(x_values)
                        min_x = min(x_values)
    
                        y_values = []
                        for vertex in block_vertices:
                            if 'y' in vertex:
                                y_values.append(vertex['y'])                         
                        max_y = max(y_values)
                        min_y = min(y_values)
    
                        block_x = max_x - min_x
                        block_y = max_y - min_y

                        num_letter = len(desc)
                        area = block_x * block_y
                        letter = area / num_letter
                        mean_letter_in_block+=letter
                    mean_letter_in_block=mean_letter_in_block/len_words
                    
                    text1=get_normal_text(text)
                    text_split=text1.split(' ')
                    flag=0
                    c=0
                    for t in text_split:
                        try:
                            float(t)
                            c=c+1  #מספר המילים שהם מספרים
                        except ValueError:
                            flag=10
                        for en in arr_entity_types:
                            if t==en:
                                c=c+1
                        for date in arr_date:
                            if t==date:
                                c=c+1
                    if c<(len(text_split)*0.7):
                        if private_ans!=0:
                            dict_mean_letter_in_block.append((text,mean_letter_in_block , private_ans))
                        else:
                            dict_mean_letter_in_block.append((text,mean_letter_in_block , ans))
        sorted_dict_mean_letter_in_block = sorted(dict_mean_letter_in_block, key=lambda x: x[1], reverse=True)
        
        if sorted_dict_mean_letter_in_block!=[]:
            min_num_of_title_letter=int((sorted_dict_mean_letter_in_block[0][1])/2)
        else:
            min_num_of_title_letter=0
            
        for i in sorted_dict_mean_letter_in_block:
            ans=i[2]-1
            if i[1]>=min_num_of_title_letter:
                dict_block_by_loc_title[ans].append((i[0]))
        print(dict_block_by_loc_title)
        return dict_block_by_loc_title
    # return sorted_dict_mean_letter_in_block               

In [27]:
import numpy as np
from PIL import Image
from skimage.transform import resize

def check_type_binding(image_path):
    # model_path = "./pic_doc_ed.h5"  # Update this with the full path to your model file
    # loaded_model = tf.keras.models.load_model("binding.h5")
    loaded_model = tf.keras.models.load_model('./classification_models/binding.h5')

    valid_images = ['jpeg', 'jpg', 'png']
    ext = image_path.split('.')[1]
    
    if ext.lower() in valid_images:
        image = Image.open(image_path)
        resized_image = resize(np.array(image), (180, 180, 3))  # Resize to match the model's input shape
        
        image_features = resized_image.flatten()
        
        # Pad or truncate the features to match the expected number
        if len(image_features) < (180 * 180 * 3):
            image_features = np.pad(image_features, (0, (180 * 180 * 3) - len(image_features)), mode='constant')
        elif len(image_features) > (180 * 180 * 3):
            image_features = image_features[:180 * 180 * 3]
        
        prediction = loaded_model.predict(image_features.reshape(1, 180, 180, 3))  # Reshape to match the model's input
        
        # Get the class with the highest probability
        predicted_class = np.argmax(prediction)
        
        if predicted_class == 0:
            return 'Not binding'
        elif predicted_class == 1:
            return 'Yad Vashem binding'
        else:
            return 'Another binding'

In [28]:
def check_type_pic_doc_ed(image_path):
# Load the saved KNN model
    # loaded_model = joblib.load('logistic_model.pkl')
    loaded_model = tf.keras.models.load_model("./classification_models/pic-doc-ed.h5")
    # Loop through the images in the folder
    valid_images = ['jpeg', 'jpg', 'png']  # Define the valid image extensions
    ext = image_path.split('.')[1]
    if ext.lower() in valid_images:
        image = Image.open(image_path)
        # Resize the image to match the expected input size of the SVM model
        resized_image = resize(np.array(image), (50, 50))  # Resize to (50, 50) or adjust as needed
    
        # Flatten and add zeros to match the expected number of features
        image_features = resized_image.flatten()
        if len(image_features) < 6750:
            image_features = np.pad(image_features, (0, 6750 - len(image_features)), mode='constant')
        elif len(image_features) > 6750:
            image_features = image_features[:6750]
    
                # Apply the loaded SVM model to the image
        prediction = loaded_model.predict(image_features.reshape(1, -1))
    
        if prediction == 0:
            return 'Testimony page'
        elif prediction == 1:
            return 'document'
        else:
            return 'Image'
    # return 'Another binding'

In [29]:

def get_data_Languages(data_response):
    x = data_response['responses'][0]['fullTextAnnotation']['pages'][0]
    if 'property' in x: 
        data_languages = data_response['responses'][0]['fullTextAnnotation']['pages'][0]["property"]["detectedLanguages"]
        lang_map={
            "de" : "German", 
            "es" :"Spanish",
            "fr" :"French", 
            "it" :"Italian", 
            "ja" :"Japanese", 
            "ko" :"Korean", 
            "ru" : "Russian", 
            "zh" :"Chinese (Mandarin)", 
            "hr" :"Croatian", 
            "nl": "Dutch", 
            'pl': 'Polish',
            'en': 'English',
            'sk': 'Slovak',
            'da': 'Danish',
            'no': 'Norwegian',
            'lt': 'Lithuanian',
            'haw': 'Hawaiian',
            'mt': 'Maltese',
            'ceb': 'Cebuano',
            'uz': 'Uzbek', 
            'ar': 'Arabic',
            'iw':'Hebrew', 
            'yi':'Yiddish', 
            'hu':'Hungarian', 
            'lb':'Luxembourgish',  
            'lv':'Latvian',
            'om':'Oromo', 
            'fy':'Frisian',
            'la':'Latin', 
            'sv' :'Swedish', 
            'af': 'Afrikaans',
            'am': 'Amharic',
            'az': 'Azerbaijani',
            'be': 'Belarusian',
            'bg': 'Bulgarian',
            'bn': 'Bengali',
            'bs': 'Bosnian',
            'ca': 'Catalan',
            'co': 'Corsican',
            'cs': 'Czech',
            'cy': 'Welsh',
            'el': 'Greek',
            'eo': 'Esperanto',
            'et': 'Estonian',
            'eu': 'Basque',
            'fa': 'Persian',
            'fi': 'Finnish',
            'fj': 'Fijian',
            'fo': 'Faroese',
            'gd': 'Scottish Gaelic',
            'gl': 'Galician',
            'gu': 'Gujarati',
            'ha': 'Hausa',
            'he': 'Hebrew',
            'hi': 'Hindi',
            'hmn': 'Hmong',
            'ht': 'Haitian Creole',
            'hy': 'Armenian',
            'id': 'Indonesian',
            'ig': 'Igbo',
            'is': 'Icelandic',
            'jv': 'Javanese',
            'ka': 'Georgian',
            'kk': 'Kazakh',
            'km': 'Khmer',
            'kn': 'Kannada',
            'ku': 'Kurdish',
            'ky': 'Kyrgyz',
            'lo': 'Lao',
            'mg': 'Malagasy',
            'mi': 'Maori',
            'mk': 'Macedonian',
            'ml': 'Malayalam',
            'mn': 'Mongolian',
            'mr': 'Marathi',
            'ms': 'Malay',
            'my': 'Burmese',
            'ne': 'Nepali',
            'ny': 'Chichewa',
            'or': 'Odia',
            'pa': 'Punjabi',
            'ps': 'Pashto',
            'pt': 'Portuguese',
            'ro': 'Romanian',
            'rw': 'Kinyarwanda',
            'sd': 'Sindhi',
            'si': 'Sinhala',
            'sl': 'Slovenian',
            'sm': 'Samoan',
                'sn': 'Shona',
            'so': 'Somali',
            'sq': 'Albanian',
            'sr': 'Serbian',
            'st': 'Sesotho',
            'su': 'Sundanese',
            'sw': 'Swahili',
            'ta': 'Tamil',
            'te': 'Telugu',
            'tg': 'Tajik',
            'th': 'Thai',
            'tk': 'Turkmen',
            'tl': 'Filipino',
            'tr': 'Turkish',
            'tt': 'Tatar',
            'ug': 'Uyghur',
            'uk': 'Ukrainian',
            'ur': 'Urdu',
            'vi': 'Vietnamese',
            'xh': 'Xhosa',
            'yo': 'Yoruba',
            'zu': 'Zulu'
        }
        for la in data_languages:
            i=la['languageCode']
            if i in lang_map:
                la['languageCode']=lang_map[i]
        sorted_languages = sorted(data_languages, key=lambda x: x['confidence'], reverse=True)
        len_sorted = len(sorted_languages)
        if len_sorted > 4:
            len_sorted = 4
        top_languages=[(la['languageCode'], la['confidence']) for la in sorted_languages[:len_sorted]]
        return top_languages

In [30]:

import csv

def write_to_csv_title(file_name):
    with open(file_name, mode='a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Image Path', 'top_title'])

def write_to_csv(data, file_name):
    with open(file_name, mode='a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow([data[0], data[1]])

def read_from_csv(file_name):
    with open(file_name, mode='r') as file:
        reader = csv.reader(file)
        for row in reader:
            print(row)
            return row

In [31]:
import re

def extract_dates_from_ocr_response(text):
    date_formats = [
        r'\b\d{1,2}[./-]\d{1,2}[./-]\d{2,4}\b',  # Date format: 28.05.1944 or 28/05/44
        r'\b\d{1,2}\.\s(?:Januar|Februar|März|April|Mai|Juni|Juli|August|September|Oktober|November|Dezember)\s\d{2,4}\b',  # Date format: 28. Mai 1944
        r'\b(?:january|february|march|april|may|june|july|august|september|october|november|december)\s+\d{1,2},?\s+\d{2,4}\b',
        r'\b\d{1,2}[./-]\s(?:Jan\.?|Feb\.?|Mar\.?|Apr\.?|May|Jun\.?|Jul\.?|Aug\.?|Sep\.?|Oct\.?|Nov\.?|Dec\.?)\s\d{2,4}\b',  # Date format: 20. Jan. 1991
        r'\b\d{1,2}\s+(?:JANUARY|FEBRUARY|MARCH|APRIL|MAY|JUNE|JULY|AUGUST|SEPTEMBER|OCTOBER|NOVEMBER|DECEMBER)\s+\d{2,4}\b',  # New date format
        r'\b\d{4}-\d{4}\b'  # Date format: 2011-2012
    ]

    dates_found = []

    for date_format in date_formats:
        date_matches = re.finditer(date_format, text, re.IGNORECASE)
        for date_match in date_matches:
            dates_found.append(date_match.group())

    return dates_found

# Example usage to identify dates in '20 JULY 2020' format and '2011-2012' format

In [32]:
def append_dates_new(entity_types):
    date_finish=[]
    for type_name, entities in entity_types.items():
        for en in entities:
            en_split=en.split(' ')
            for e_split in en_split:
                date_found=extract_dates_from_ocr_response(e_split)
                if len(date_found)>0:
                    flag=False
                    date=date_found[0]
                    for d in date_finish:
                        if d==date:
                            flag=True
                    if flag==False:
                        date_finish.append(date)
    return(date_finish)

In [33]:
def get_title_another_binding(json, json_res, date_arr):
    [loc_title_up,loc_title_middle,loc_title_down ]=get_size_y(json_res)
    if [loc_title_up,loc_title_middle,loc_title_down ]!=[(0, 0), (0, 0), (0,0,)]:
        fullTextAnnotation_pages = json['responses'][0]['fullTextAnnotation']['pages']
        word_count=get_num_words_in_json(json) 
        sorted_block_areas_num=get_big_letter_from_desc(json,word_count )
        dict_block_by_loc_title={0:[], 1:[], 2:[]}
        dict_mean_letter_in_block=[]
        private_ans=0
        for page in fullTextAnnotation_pages:
            blocks = page['blocks']
            if (len(blocks)<=3):
                private_ans=2
            for block in blocks:
                words=block['paragraphs'][0]['words']
                len_words=len(words)
                if len_words<=15:
                    mean_letter_in_block=0
                    text=get_text_from_block(block)
                    block_vertices = block['boundingBox']['vertices']
                    y=block_vertices[3]['y']-block_vertices[0]['y']
                    if block_vertices[0]['y']<=loc_title_up[1]:
                        ans=1
                    elif block_vertices[0]['y']<=loc_title_middle[1]:
                        ans=2
                    else:
                        ans=3
                    for word in words:
                        desc=''
                        word1=word['symbols']
                        for l in word1:
                            desc+=l['text']
                        word_vertices=word['boundingBox']['vertices']
                        x_values = []
                        for vertex in block_vertices:
                            if 'x' in vertex:
                                x_values.append(vertex['x'])
                        max_x = max(x_values)
                        min_x = min(x_values)
    
                        y_values = []
                        for vertex in block_vertices:
                            if 'y' in vertex:
                                y_values.append(vertex['y'])                         
                        max_y = max(y_values)
                        min_y = min(y_values)
    
                        block_x = max_x - min_x
                        block_y = max_y - min_y

                        num_letter = len(desc)
                        area = block_x * block_y
                        letter = area / num_letter
                        mean_letter_in_block+=letter
                    mean_letter_in_block=mean_letter_in_block/len_words
                    
                    text1=get_normal_text(text)
                    text_split=text1.split(' ')
                    flag=0
                    c=0
                    for t in text_split:
                        try:
                            float(t)
                            c=c+1  #מספר המילים שהם מספרים
                        except ValueError:
                            flag=10
                        for en in arr_entity_types:
                            if t==en:
                                c=c+1
                        for date in arr_date:
                            if t==date:
                                c=c+1
                    if c<(len(text_split)*0.7):
                        if private_ans!=0:
                            dict_mean_letter_in_block.append((text,mean_letter_in_block , private_ans))
                        else:
                            dict_mean_letter_in_block.append((text,mean_letter_in_block , ans))
    sorted_dict_mean_letter_in_block = sorted(dict_mean_letter_in_block, key=lambda x: x[1], reverse=True)
    return sorted_dict_mean_letter_in_block[2:]
        # if sorted_dict_mean_letter_in_block!=[]:
        #     min_num_of_title_letter=int((sorted_dict_mean_letter_in_block[0][1])/2)
        # else:
        #     min_num_of_title_letter=0
            
        # for i in sorted_dict_mean_letter_in_block:
        #     ans=i[2]-1
        #     if i[1]>=min_num_of_title_letter:
        #         dict_block_by_loc_title[ans].append((i[0]))
        # print(dict_block_by_loc_title)
        # return dict_block_by_loc_title
    # return sorted_dict_mean_letter_in_block               

In [34]:
def get_date_from_Testimony_page(json, format_json, json_res):
    date_arr=[]
    date=''
    date1=extract_dates_from_ocr_response(format_json['full_desc_with_verices'][0]['text'])
    if len(date1)>0:
        date_arr=date1
    entity_types=format_json['entity_types']
    if 'DATE' in entity_types:
        date2=entity_types['DATE']
        for date in date2:
            if date not in date_arr:
                date_arr.append(date)
    if 'CARDINAL' in entity_types:
        for car in entity_types['CARDINAL']:
            d=extract_dates_from_ocr_response(car)
            if len(d)>0:
                if d[0] not in date_arr:
                    date_arr.append(d[0])
    date3=append_dates_new(entity_types)
    if len(date3)>0:
        for date in date3:
            if date not in date_arr:
                date_arr.append(date)
    if len(date_arr)>0:
        date_arr=list(set(date_arr))
        res_date =  get_date_Testimony(json_res, json, date_arr, format_json)
        if len(res_date)>0:
            date= res_date  
    return date_arr, date   

In [35]:
def get_date_from_document( json, format_json, json_res):
    date_arr=[]
    date=''
    date1=extract_dates_from_ocr_response(format_json['full_desc_with_verices'][0]['text'])
    if len(date1)>0:
        date_arr=date1
    entity_types=format_json['entity_types']
    if 'DATE' in entity_types:
        date2=entity_types['DATE']
        for date in date2:
            d=extract_dates_from_ocr_response(date)
            if len(d)>0:
                if d[0] not in date_arr:
                    date_arr.append(d[0])
    if 'CARDINAL' in entity_types:
        for car in entity_types['CARDINAL']:
            d=extract_dates_from_ocr_response(car)
            if len(d)>0:
                if d[0] not in date_arr:
                    date_arr.append(d[0])
    date3=append_dates_new(entity_types)
    if len(date3)>0:
        for date in date3:
            d=extract_dates_from_ocr_response(date)
            if len(d)>0:
                if d[0] not in date_arr:
                    date_arr.append(d[0])
    if len(date_arr)>0:
        date_arr=list(set(date_arr))
        res_date =  get_date(json_res, json, date_arr, format_json)
        if len(res_date)>0:
            date= res_date  
    return date_arr, date

In [36]:
def create_format_json(img_path, json, format_json):
    page_type=format_json['page type']
    format_json['data_Languages'] = get_data_Languages(json)
    json_res = create_json_get_by_y(json)
    if json_res:
        format_json['full_desc_with_verices'] = json_res.get('full_desc_with_verices', [])
        format_json['words'] = json_res.get('words', [])
        text_ = format_json['full_desc_with_verices'][0].get('text', '')
        text_ = ' '.join(text_.split("\n")) if text_ else ''
        format_json['full_desc_with_verices'][0]['text'] = text_
        entity_types = get_finds_dates_places_persons(text_)
        format_json['entity_types']=entity_types 
        if page_type=='another binding':
            print("ddddddddddddddddddd")
            title = get_title_another_binding(json, json_res, date_arr)
            if title:
                print("title", title)
                format_json['top_title'] = title
                return format_json
        if page_type=='yad vashem binding':
            print("ddddddddddddddddddd")
            format_json['top_title']=['YAD VASHEM ARCHIVES', 'ארכיון יד ושם']
            return format_json
        if page_type=='Testimony page':
            format_json['top_title']=['page of Testimony', 'דף עד']
            date_arr, date=get_date_from_Testimony_page(json, format_json,json_res )
            print(date)
            format_json['Production date']=date
            return format_json
        if  page_type=='Testimony page including a photograph':
            format_json['top_title']=['page of Testimony', 'דף עד']
            date_arr, date=get_date_from_Testimony_page(json, format_json,json_res )
            print(date)
            format_json['Production date']=date
            return format_json
        if page_type=='document':
            date_arr, date=get_date_from_document(json, format_json,json_res )
            format_json['Production date']=date
            dict_block_by_loc_title = get_title(json, json_res, date_arr)
            if dict_block_by_loc_title:
                title = dict_block_by_loc_title[0]
                if len(title)>3:
                    title=title[:3]
                format_json['top_title'] = title
                return format_json
    return format_json

In [37]:
import json

def read_json_file(json_filename):
    try:
        with open(json_filename, 'r', encoding='utf-8') as json_file:
            data = json.load(json_file)
        return data
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON: {e}")
        return None

In [38]:
def process_images_in_folders(main_folder):
    valid_images = ['.json']  # Define the valid image extensions
    imgs_dict = {}
    regular_imgs=[]
    pictures=[]
    to_remove=[]
    for folder in os.listdir(main_folder):
        folder_path = os.path.join(main_folder, folder)
        if not os.path.isdir(folder_path):
            continue
        for image_file in os.listdir(folder_path):
            ext = os.path.splitext(image_file)[1]
            image_path = os.path.join(folder_path, image_file)
            if ext.lower() not in valid_images:
                type_pred=check_type_page(image_path)
                if type_pred=='picture':
                    pictures.append(image_path)
                else:
                    regular_imgs.append(image_path)
            else:
                json=read_json_file(image_path)
                if json!=None:
                    imgs_dict[image_path]=json
                else:
                    to_remove.append(image_path.split('.')[0])
    regular_imgs_2=[]
    for i in regular_imgs:
        ii=i.split('.')[0]
        if ii not in to_remove:
            regular_imgs_2.append(i)
    return imgs_dict, regular_imgs_2, pictures

In [39]:
import json

def write_for_json(json_filename, json_object):
    with open(json_filename, 'w', encoding='utf-8') as json_file:
        json.dump(json_object, json_file, ensure_ascii=False)

In [40]:
def predict_ed(class_names, img_height, img_width, pic_path, loaded_model):
    img = tf.keras.utils.load_img(pic_path, target_size=(img_height, img_width))
    img_array = tf.keras.utils.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0) # Create a batch
    predictions = loaded_model.predict(img_array)
    
    return class_names[np.argmax(predictions[0])]


In [41]:

def predict_type_ed(image):
    loaded_model = tf.keras.models.load_model("./classification_models/ed.h5")
    class_names = ['ed', 'edp']
    img_height = 180
    img_width = 180
    if image.endswith(('.jpg', '.JPG', '.png', '.gif')):
        pred = predict_ed(class_names, img_height, img_width, image, loaded_model)
        return pred

In [42]:
def predict_pic_doc_ed(class_names, img_height, img_width, pic_path, loaded_model):
    img = tf.keras.utils.load_img(pic_path, target_size=(img_height, img_width))
    img_array = tf.keras.utils.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0) # Create a batch
    predictions = loaded_model.predict(img_array)
    
    return class_names[np.argmax(predictions[0])]


In [43]:
def predict_type_pic_doc_ed(image):
    loaded_model = tf.keras.models.load_model("./classification_models/pic-doc-ed.h5")
    class_names = ['pictures', 'documents', 'ed']
    img_height = 180
    img_width = 180
    if image.endswith(('.jpg', '.JPG', '.png', '.gif')):
        pred = predict_pic_doc_ed(class_names, img_height, img_width, image, loaded_model)
        return pred

In [44]:
def predict_binding(class_names, img_height, img_width, pic_path, loaded_model):
    img = tf.keras.utils.load_img(pic_path, target_size=(img_height, img_width))
    img_array = tf.keras.utils.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0) # Create a batch
    predictions = loaded_model.predict(img_array)
    return class_names[np.argmax(predictions[0])]


In [45]:

def predict_type_binding(image):
    loaded_model = tf.keras.models.load_model("./classification_models/binding.h5")
    class_names = ['not binding', 'yad vashem binding', 'another binding']
    img_height = 180
    img_width = 180
    if image.endswith(('.jpg', '.JPG', '.png', '.gif')):
        pred = predict_binding(class_names, img_height, img_width, image, loaded_model)
        return pred

In [46]:
def check_type_page(image_path):
    pred_pic_doc_ed=predict_type_pic_doc_ed(image_path)
    if pred_pic_doc_ed=='pictures':
        return 'picture'
    elif pred_pic_doc_ed=='documents':
        pred_bind=predict_type_binding(image_path)
        if pred_bind=='not binding':
            return 'document'
        else:
            pred_bind
    else:
        pred_ed=predict_type_ed(image_path)
        if pred_ed=='ed':
            return 'Testimony page'
        else :
            return 'Testimony page including a photograph'

In [47]:
def write_to_sql(json_data):
    image_name=json_data.get("image_path", '')
    full_desc_with_verices=json_data.get("full_desc_with_verices", '')
    lang_text_1=''
    lang_text_1+=str(full_desc_with_verices[0])
    lang_text_1+=','
    lang_text_1+=str(full_desc_with_verices[1])
    full_desc_with_verices=lang_text_1
    words=json_data.get("words", '')
    flat_data = [item for sublist in words for item in sublist]
    long_string = ', '.join(map(str, flat_data))
    words=long_string
    top_title=json_data.get("top_title", '')
    format_json_path=json_data.get("format_json_path", '')
    json_path=json_data.get("json_path", '')
    date=json_data.get("Production date", '')
    if date==[]:
        date=''
    type_page=json_data.get("type_page", '')
    entity_types=str(json_data.get("entity_types", ''))
# Establish connection to the SQL Server database
    connection_string = 'DRIVER={SQL Server};SERVER=DESKTOP-E0FAPSB\SQLEXPRESS;DATABASE=Practicom-DB-format_json;Trusted_Connection=yes;'
    connection = pyodbc.connect(connection_string)
    cursor = connection.cursor()
    
    try:
        # Insert data into the Master table
        cursor.execute("INSERT INTO Master (IMG_NAME, FULL_DESC_WITH_VERTICES, WORDS, DATE, JSON_PATH, FORMAT_JSON_PATH, TYPE_PAGE, ENTITY_YTPES) VALUES (?, ?, ?, ?, ?, ?, ?)", 
                           (image_name, full_desc_with_verices, words, date, json_path, format_json_path, type_page, entity_types))
        connection.commit()
    
        # Get the generated DOC_ID
        cursor.execute("SELECT @@IDENTITY AS LastIdentity")
        doc_id = cursor.fetchone().LastIdentity
        # Insert data into the Title table with the correct DOC_ID
        for title_text in top_title:
            cursor.execute("INSERT INTO Title (DOC_ID, TITLE_TEXT) VALUES (?, ?)", (doc_id, title_text))
        connection.commit()
    
        # Insert data into the Data_Languages table
        data_languages = json_data.get("data_Languages", [])
        for language, confidence in data_languages:
            cursor.execute("INSERT INTO Data_Languages (DOC_ID, LANGUAGE, CONFIDENCE) VALUES (?, ?, ?)", (doc_id, language, confidence))
        connection.commit()
    
    except Exception as e:
        # Handle any exceptions that may occur during the database operations
        print("An error occurred:", e)
    finally:
        cursor.close()
        connection.close()

In [48]:
def write_to_sql_pictures(picture):
    connection_string = 'DRIVER={SQL Server};SERVER=DESKTOP-E0FAPSB\SQLEXPRESS;DATABASE=Practicom-DB-format_json;Trusted_Connection=yes;'
    connection = pyodbc.connect(connection_string)
    cursor = connection.cursor()
    
    try:
        # Insert data into the Master table
        cursor.execute("INSERT INTO Pictures (IMG_PATH) VALUES (?)", 
                           (picture))
        connection.commit()
    except Exception as e:
# Handle any exceptions that may occur during the database operations
        print("An error occurred:", e)
    finally:
        cursor.close()
        connection.close()

In [49]:
import ast
import json
def second_steps_for_all_images(data):
    new_json_dict={}
    regular_imgs=[]
    imgs_dict, regular_imgs, pictures=process_images_in_folders(data)
    img=''
    print(len(imgs_dict), len(regular_imgs), len(pictures))
    for picture in pictures:
        print("pic")
        # write_to_sql_pictures(picture)
    for img_path, json in imgs_dict.items():
        format_json={}
        format_json['page type']=check_type_page(img_path)
        format_json=create_format_json(img_path, json, format_json)
        format_json['json_path']=img_path
        img=img_path.split('.')[0]
        new_json_dict[img]=format_json
    for img_path in regular_imgs:
        format_json['image_path']=img_path
        img1=img_path.split('.')[0]
        format_json=new_json_dict[img1]
        format_json['format_json_path']=img1+'_format.json'
        # write_to_sql(format_json)
        # write_for_json(format_json['format_json_path'], format_json)
        # print(img_path, format_json.get('Production date', 0), '  :::  ', format_json.get('top_title', ''))
        # print("img_path", img_path)

In [50]:
data='data_practucom/test'
second_steps_for_all_images(data)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step


...

(47, 127.0)
30.8.45


(19, 201.25)
10-30781-1


(12, 223.25)
Monday


(15, 153.0)
8. april 1946


(150, 931.75)
3.4.1955
(23, 177.75)
23531
